In [1]:
from tqdm import tqdm
from dumper.utils import gsheet_to_df
from apis_core.utils import get_object_from_pk_or_uri
import pandas as pd

In [2]:
col, _ = Collection.objects.get_or_create(name="Schnitzler-Theater")
temp_col, _ = Collection.objects.get_or_create(name="temp-issue-227")

In [3]:
df = gsheet_to_df("1NAyAetEWR1RzyGcdTWc0KGK2ip19PpSyIOBBCOnyRnA")

200


In [4]:
df

,start-date,end-date,start-date-written,end-date-written,title,eventType
0,1876-03-15,1876-03-15,1876-03-15,1876-03-15,Aufführung von Das heiß Eysen; Die ehrlich Bäc...,1200
1,1876-04-02,1876-04-02,1876-04-02,1876-04-02,"Gesellschaftskonzert, 2.4.1876",4302
2,1876-04-27,1876-04-27,1876-04-27,1876-04-27,"Aufführung von Götz von Berlichingen, 27.4.1876",1200
3,1876-05-08,1876-05-08,1876-05-08,1876-05-08,Aufführung von Uriel Acosta. Trauerspiel in fü...,1200
4,1876-06-29,1876-06-29,1876-06-29,1876-06-29,"Aufführung von Die Hugenotten, 29.6.1876",1200
...,...,...,...,...,...,...
3905,1931-10-09,1931-10-09,1931-10-09,1931-10-09,"Filmvorführung von Razzia auf Liebe, 9.10.1931",4234
3906,1931-10-11,1931-10-11,1931-10-11,1931-10-11,"Filmvorführung von Eva, 11.10.1931",4234
3907,1931-10-12,1931-10-12,1931-10-12,1931-10-12,"Filmvorführung von Purpur und Waschblau, 12.10...",4234
3908,1931-10-16,1931-10-16,1931-10-16,1931-10-16,"Filmvorführung von Der König der Blitzer, 16.1...",4234


In [5]:
pmb_ids = []
for i, row in tqdm(df.iterrows(), total=len(df)):
    event_type = EventType.objects.get(id=row["eventType"].split(" ")[0])
    item, _ = Event.objects.get_or_create(
        name=row["title"],
        start_date_written=row["start-date"],
        kind=event_type
    )
    item.collection.add(col)
    item.collection.add(temp_col)
    pmb_ids.append([i, f"{item.start_date}", item.name, item.id])
                                       

100%|███████████████████████████████████████████████████████████████████████| 3910/3910 [12:54<00:00,  5.05it/s]


In [6]:
new_df = pd.DataFrame(pmb_ids)

In [7]:
df["pmb_ids"] = new_df[3]

In [8]:
df.to_csv("events-pmb_ids.csv", index=False)